## **Datos Economía**

In [4]:
import pandas as pd
import numpy as np
import os
import sys

In [5]:
municipios_cluster = pd.read_csv(r"external_data/municipios_con_cluster.csv", sep=",")

### **pensionistas_100**

In [72]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
PENSIONISTAS_CSV = "data/economia/pensionistas.csv"    # sep=";"

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)' para casar con pensionistas/cluster
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

# Filtrar Sexo Total
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

# Normalizar y parsear
pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
# Columnas a string (años)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN PENSIONISTAS (ya ancho)
# ============================================================
pens = pd.read_csv(PENSIONISTAS_CSV, sep=";", dtype=str)

pens["Nombre"] = pens["Nombre"].apply(normalize_nombre)
pens["key"] = pens["Nombre"].apply(canon_key)

# Detectar columnas año
year_cols = [c for c in pens.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    pens[c] = pens[c].apply(parse_es_number)

pensionistas_total = pens[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
#     Debe tener columnas: Nombre, Cluster (y otras)
# ============================================================
# Ejemplo: municipios_cluster = pd.read_csv("ruta/a/cluster.csv", sep=",")
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs pensionistas)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_pens = {int(c) for c in year_cols}
years_to_process = sorted(years_pobl.intersection(years_pens))

print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Selección por año
    numero_pensionistas = (
        pensionistas_total[["key", "Nombre", col]]
        .rename(columns={col: "pensionistas_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    # Merge pens + pobl por key
    datos = pd.merge(numero_pensionistas, poblacion_anio, on="key", how="inner")

    # Añadir cluster (y si quieres usar el nombre del cluster como salida)
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    # Usar Nombre del cluster si existe; si no, mantener el de pensionistas
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["pensionistas_total"] = pd.to_numeric(datos["pensionistas_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # pensionistas por 100 habitantes
    datos["pensionistas_100"] = ((datos["pensionistas_total"] / datos["total_poblacion"]) * 100).fillna(0)

    # 0 => "menor es mejor" (como en tu lógica original)
    datos["type"] = 0

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["pensionistas_100"].mean()
        desviacion = grupo["pensionistas_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Si hay municipios sin cluster, no entran en groupby.
    # Puedes decidir qué hacer: aquí los dejamos fuera del cálculo por cluster y luego los añadimos tal cual.
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    # Para los sin cluster, dejamos atractividad_0_100 = 0 (o lo que prefieras)
    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(columns={"atractividad_0_100": "pensionistas_100"})

    # Guardar
    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}pensionistas_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2023, 2024, 2025]
Archivo del 2023 exportado a data_interfaz/economia/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/economia/normalizacion_final/2024/
Archivo del 2025 exportado a data_interfaz/economia/normalizacion_final/2025/


C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\2962386376.py:180: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\2962386376.py:180: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **ocupadosColectivos_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# ============================================================
# HELPERS (igual que antes)
# ============================================================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # sin tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s


# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
OCUPADOS_CSV = "data/economia/ocupados_colectivos.csv"    # sep=";"


# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

# Filtrar Sexo Total
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

# Normalizar y parsear
pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
# Columnas a string (años)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ------------------------------------------------------------
# Preparar CLUSTERS (solo una vez)
# ------------------------------------------------------------
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ------------------------------------------------------------
# Preparar OCUPADOS (solo una vez)
# ------------------------------------------------------------

# ============================================================
# 2) CARGA Y PREPARACIÓN PENSIONISTAS (ya ancho)
# ============================================================
ocupados_totales = pd.read_csv(OCUPADOS_CSV, sep=";", dtype=str)

ocupados_totales["Nombre"] = ocupados_totales["Nombre"].apply(normalize_nombre)
ocupados_totales["key"] = ocupados_totales["Nombre"].apply(canon_key)

# Detectar columnas año
year_cols_ocu = [c for c in ocupados_totales.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_ocu:
    ocupados_totales[c] = ocupados_totales[c].apply(parse_es_number)

pensionistas_total = ocupados_totales[["key", "Nombre"] + year_cols_ocu].copy()

ocupados_totales = ocupados_totales.copy()  # <- tu DF
ocupados_totales["Nombre"] = ocupados_totales["Nombre"].apply(normalize_nombre)
ocupados_totales["key"] = ocupados_totales["Nombre"].apply(canon_key)

# ------------------------------------------------------------
# Años a procesar: intersección (ocupados vs población)
# ------------------------------------------------------------
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_ocu = {int(c) for c in year_cols_ocu}
years_to_process = sorted(years_pobl.intersection(years_ocu))

print("Años a procesar (ocupadosColectivos_100):", years_to_process)

# ============================================================
# FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_ocupados = (
        ocupados_totales[["key", "Nombre", col]]
        .rename(columns={col: "ocupados_totales"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_ocupados, poblacion_anio, on="key", how="inner")

    # Añadir cluster y (opcional) el nombre del cluster como salida
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["ocupados_totales"] = pd.to_numeric(datos["ocupados_totales"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # ocupados por 100 habitantes
    datos["ocupadosColectivos_100"] = ((datos["ocupados_totales"] / datos["total_poblacion"]) * 100).fillna(0)

    # type=1 => "más es mejor"
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["ocupadosColectivos_100"].mean()
        desviacion = grupo["ocupadosColectivos_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Si falta cluster: los dejamos con 0 (o cambia esta lógica si prefieres media global)
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "ocupadosColectivos_100"}
    )

    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)

    resultado.to_csv(f"{output_path}ocupadosColectivos_100.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# PROCESAR TODOS LOS AÑOS DISPONIBLES
# ============================================================
for y in years_to_process:
    procesar_anio(y)


### **contratos_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# ============================================================
# HELPERS (igual que antes)
# ============================================================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # Quita código INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # Sin tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
CONTRATOS_CSV = "data/economia/contratos_total.csv"          # sep=";"

# ============================================================
# 1) CARGA Y PREPARACIÓN DE LA POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN DE CONTRATOS (ya en ancho)
# ============================================================
contratos_total = pd.read_csv(CONTRATOS_CSV, sep=";", dtype=str)
contratos_total["Nombre"] = contratos_total["Nombre"].apply(normalize_nombre)
contratos_total["key"] = contratos_total["Nombre"].apply(canon_key)

year_cols_contratos = [c for c in contratos_total.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_contratos:
    contratos_total[c] = contratos_total[c].apply(parse_es_number)

# ============================================================
# 3) CARGA DE CLUSTERS
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# AÑOS A PROCESAR
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_contratos = {int(c) for c in year_cols_contratos}
years_to_process = sorted(years_pobl.intersection(years_contratos))

print("Años a procesar (contratos_100):", years_to_process)

# ============================================================
# FUNCIÓN DE PROCESO POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    contratos = contratos_total[["key", "Nombre", col]].rename(columns={col: "contratos_total"})
    poblacion = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(contratos, poblacion, on="key", how="inner")
    datos = pd.merge(datos, municipios_cluster_ren[["key", "Nombre", "cluster"]],
                     on="key", how="left", suffixes=("", "_cluster"))

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["contratos_total"] = pd.to_numeric(datos["contratos_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")
    datos["contratos_100"] = ((datos["contratos_total"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["contratos_100"].mean()
        desviacion = grupo["contratos_100"] - media
        max_dev = desviacion.abs().max()
        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "contratos_100"}
    )

    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}contratos_100.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# EJECUTAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (contratos_100): [2023, 2024]
Archivo del 2023 exportado a data_interfaz/economia/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/economia/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\3948618419.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\3948618419.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **pensionMedia**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# ============================================================
# HELPERS
# ============================================================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita código INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # sin tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"             # sep=";"
PENSION_MEDIA_CSV = "data/economia/pension_media.csv"           # sep=";"
CLUSTERS_CSV = "external_data/municipios_cluster.csv"           # sep=";"

# ============================================================
# 1) CARGA Y PREPARACIÓN DE POBLACIÓN (largo → ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN DE PENSIONES (ya en ancho)
# ============================================================
pension_media = pd.read_csv(PENSION_MEDIA_CSV, sep=";", dtype=str)
pension_media["Nombre"] = pension_media["Nombre"].apply(normalize_nombre)
pension_media["key"] = pension_media["Nombre"].apply(canon_key)

year_cols = [c for c in pension_media.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    pension_media[c] = pension_media[c].apply(parse_es_number)

# ============================================================
# 3) CARGA DE CLUSTERS
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# AÑOS A PROCESAR
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_pens = {int(c) for c in year_cols}
years_to_process = sorted(years_pobl.intersection(years_pens))

print("Años a procesar (pension_media):", years_to_process)

# ============================================================
# FUNCIÓN DE PROCESO POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    df_pension = pension_media[["key", "Nombre", col]].rename(columns={col: "pension_media"})
    df_pobl = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(df_pension, df_pobl, on="key", how="inner")
    datos = pd.merge(datos, municipios_cluster_ren[["key", "Nombre", "cluster"]],
                     on="key", how="left", suffixes=("", "_cluster"))

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["pension_media"] = pd.to_numeric(datos["pension_media"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")
    datos["pension_media"] = ((datos["pension_media"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 0  # menos es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["pension_media"].mean()
        desviacion = grupo["pension_media"] - media
        max_dev = desviacion.abs().max()
        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "pension_media"}
    )

    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}pension_media.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# EJECUCIÓN
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (pension_media): [2023, 2024, 2025]
Archivo del 2023 exportado a data_interfaz/economia/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/economia/normalizacion_final/2024/
Archivo del 2025 exportado a data_interfaz/economia/normalizacion_final/2025/


C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\958505458.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\958505458.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at

### **paro_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# ============================================================
# HELPERS
# ============================================================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita código INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # sin tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"
PARO_CSV = "data/economia/paro_total.csv"
CLUSTERS_CSV = "external_data/municipios_cluster.csv"

# ============================================================
# 1) CARGA Y PREPARACIÓN DE POBLACIÓN (largo → ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN DE PARO TOTAL (ya en ancho)
# ============================================================
paro_total = pd.read_csv(PARO_CSV, sep=";", dtype=str)
paro_total["Nombre"] = paro_total["Nombre"].apply(normalize_nombre)
paro_total["key"] = paro_total["Nombre"].apply(canon_key)

year_cols = [c for c in paro_total.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    paro_total[c] = paro_total[c].apply(parse_es_number)

# ============================================================
# 3) CARGA DE CLUSTERS
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# AÑOS A PROCESAR
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_paro = {int(c) for c in year_cols}
years_to_process = sorted(years_pobl.intersection(years_paro))

print("Años a procesar (paro_total):", years_to_process)

# ============================================================
# FUNCIÓN DE PROCESO POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(anio)

    df_paro = paro_total[["key", "Nombre", col]].rename(columns={col: "paro_total"})
    df_pobl = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(df_paro, df_pobl, on="key", how="inner")
    datos = pd.merge(datos, municipios_cluster_ren[["key", "Nombre", "cluster"]],
                     on="key", how="left", suffixes=("", "_cluster"))

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["paro_total"] = pd.to_numeric(datos["paro_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")
    datos["paro_total"] = ((datos["paro_total"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 0  # menos es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["paro_total"].mean()
        desviacion = grupo["paro_total"] - media
        max_dev = desviacion.abs().max()
        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "paro_total"}
    )

    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}paro_total.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# EJECUCIÓN
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (paro_total): [2023, 2024, 2025]
Archivo del 2023 exportado a data_interfaz/economia/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/economia/normalizacion_final/2024/
Archivo del 2025 exportado a data_interfaz/economia/normalizacion_final/2025/


C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\2007029807.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\2007029807.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **renta_bruta**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# ============================================================
# HELPERS
# ============================================================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"
RENTA_CSV = "data/economia/renta_bruta.csv"

# ============================================================
# 0) CARGA Y PREPARACIÓN DE POBLACIÓN (largo → ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 1) CARGA Y PREPARACIÓN DE RENTA BRUTA
# ============================================================
renta_bruta = pd.read_csv(RENTA_CSV, sep=";", dtype=str)
renta_bruta["Nombre"] = renta_bruta["Nombre"].apply(normalize_nombre)
renta_bruta["key"] = renta_bruta["Nombre"].apply(canon_key)

year_cols_renta = [c for c in renta_bruta.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_renta:
    renta_bruta[c] = renta_bruta[c].apply(parse_es_number)

# ============================================================
# 2) AÑOS A PROCESAR (intersección renta y población)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_renta = {int(c) for c in year_cols_renta}
years_to_process = sorted(years_pobl.intersection(years_renta))

print("Años a procesar (renta_bruta):", years_to_process)

# ============================================================
# 3) CARGA Y PREPARACIÓN DE CLUSTERS
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) FUNCIÓN DE PROCESO POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(anio)

    datos_renta = renta_bruta[["key", "Nombre", col]].rename(columns={col: "renta_bruta"})
    datos_pobl = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(datos_renta, datos_pobl, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["renta_bruta"] = pd.to_numeric(datos["renta_bruta"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Renta bruta per cápita
    datos["renta_bruta"] = (datos["renta_bruta"] / datos["total_poblacion"]).fillna(0)

    # Más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["renta_bruta"].mean()
        desviacion = grupo["renta_bruta"] - media
        max_dev = desviacion.abs().max()
        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "renta_bruta"}
    )

    output_path = f"data_interfaz/economia/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)

    resultado.to_csv(f"{output_path}renta_bruta.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 5) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (renta_bruta): [2023]
Archivo del 2023 exportado a data_interfaz/economia/normalizacion_final/2023/


C:\Users\aitan\AppData\Local\Temp\ipykernel_31928\1557571753.py:143: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
